In [2]:
# Import libraries
import os
import wget
import folium
import pandas as pd
from ast import literal_eval


# Available free tiles defaults (might be possible to get more)
tiles_style = ["OpenStreetMap",
               "Stamen Terrain", 
               "Stamen Toner", 
               "Stamen Watercolor", 
               "CartoDB positron", 
               "CartoDB dark_matter"]


# Color of the Paris Metro lines
color_table = pd.read_table('./data/color_metro_paris.txt', sep=',', header=None)

# Files containing the position of the Paris metro stations and lines
filename_gares = './data/emplacement-des-gares-idf.csv'
filename_lignes = './data/traces-du-reseau-ferre-idf.csv'

# Paris
m = folium.Map(location=[48.859553, 2.336332], 
               tiles=tiles_style[5],
               zoom_start=12, 
               control_scale=True, 
               prefer_canvas=True)

# output filename
outfp = "paris_metro.html"

# We create a feature group (layer) for each set line
feature_group1 = folium.FeatureGroup(name='Line 1')
feature_group2 = folium.FeatureGroup(name='Line 2')
feature_group3 = folium.FeatureGroup(name='Line 3')
feature_group4 = folium.FeatureGroup(name='Line 3b')
feature_group5 = folium.FeatureGroup(name='Line 4')
feature_group6 = folium.FeatureGroup(name='Line 5')
feature_group7 = folium.FeatureGroup(name='Line 6')
feature_group8 = folium.FeatureGroup(name='Line 7')
feature_group9 = folium.FeatureGroup(name='Line 7b')
feature_group10 = folium.FeatureGroup(name='Line 8')
feature_group11 = folium.FeatureGroup(name='Line 9')
feature_group12 = folium.FeatureGroup(name='Line 10')
feature_group13 = folium.FeatureGroup(name='Line 11')
feature_group14 = folium.FeatureGroup(name='Line 12')
feature_group15 = folium.FeatureGroup(name='Line 13')
feature_group16 = folium.FeatureGroup(name='Line 14')

# And a dictionary associating a line string to a group
dct = {'1': feature_group1, 
       '2': feature_group2, 
       '3': feature_group3,
       '3b': feature_group4,
       '4': feature_group5,
       '5': feature_group6,
       '6': feature_group7,
       '7': feature_group8,
       '7b': feature_group9,
       '8': feature_group10,
       '9': feature_group11,
       '10': feature_group12,
       '11': feature_group13,
       '12': feature_group14,
       '13': feature_group15,
       '14': feature_group16       
      }

# Get the lines
df = pd.read_table(filename_lignes, sep=';')
# only the metro ones
df_M = df.loc[df['METRO']==1]

for index, row in df_M.iterrows():
    # which line it is
    line_str = row['INDICE_LIG']
    # get the right color
    color_metro_line = color_table.iat[color_table.loc[color_table[0]==line_str].index.values[0],1]
    
    # get the lines data and format it properly
    data = literal_eval(row['Geo Shape'])
    points = []
    for point in data['coordinates']:
        points.append(tuple([point[1], point[0]]))
    
    # plot it
    folium.PolyLine(points, 
                    color=color_metro_line,
                    weight=2,
                    opacity=1).add_to(dct[line_str])


# Get the stations
df = pd.read_table(filename_gares, sep=';')
# only the metro ones
df_M = df.loc[df['METRO']==1]

for index, row in df_M.iterrows():
    # which lines pass through
    lines = row['RES_COM'].split(' / ')
    # in case there are more just get the first
    line_str = lines[0].split('M')[1]
    
    # Get the station name
    popup_text = '{}'.format(row['NOM_GARE'].replace("'","\\'"))
    
    # which line it is
    color_metro_line = color_table.iat[color_table.loc[color_table[0]==line_str].index.values[0],1]
    color_metro_line_fill = color_metro_line
    rad = 3

    # plot it
    folium.CircleMarker(literal_eval(row['Geo Point']),
                        color=color_metro_line,
                        radius=rad,
                        fill=True,
                        fill_color=color_metro_line_fill,
                        fill_opacity=1,
                        popup=popup_text,
                        ).add_to(dct[line_str])

# add the layers to the map
feature_group1.add_to(m)
feature_group2.add_to(m)
feature_group3.add_to(m)
feature_group4.add_to(m)
feature_group5.add_to(m)
feature_group6.add_to(m)
feature_group7.add_to(m)
feature_group8.add_to(m)
feature_group9.add_to(m)
feature_group10.add_to(m)
feature_group11.add_to(m)
feature_group12.add_to(m)
feature_group13.add_to(m)
feature_group14.add_to(m)
feature_group15.add_to(m)
feature_group16.add_to(m)

# enable layer control
folium.LayerControl().add_to(m)

m.save(outfp)
m